In [1]:
import arcpy
from archaic import FeatureClass

arcpy.env.workspace = r".data\canada.geodatabase"  # type: ignore

In [3]:
fc = FeatureClass(
    "main.mines_pt",
    objectid="OBJECTID",
    name_e="NAME_E",
    type_e="TYPE_E",
    shape="SHAPE",
)

fc.get(4), fc.info.__dict__


(namespace(objectid=4,
           name_e='Raglan',
           NOM_F='Raglan',
           SOURCE='900A',
           PRV_CODE=24,
           CARTO_UID=703,
           type_e='Nickel',
           TYPE_F='Nickel',
           shape=(1584508.9825000018, 601081.8022000007)),
 {'model': types.SimpleNamespace,
  'has_parameterless_constructor': True,
  'catalog_path': '.data\\canada.geodatabase\\main.mines_pt',
  'properties': {'objectid': 'OBJECTID',
   'name_e': 'NAME_E',
   'NOM_F': 'NOM_F',
   'SOURCE': 'SOURCE',
   'PRV_CODE': 'PRV_CODE',
   'CARTO_UID': 'CARTO_UID',
   'type_e': 'TYPE_E',
   'TYPE_F': 'TYPE_F',
   'shape': 'Shape'},
  'edit_properties': {'objectid': 'OBJECTID',
   'name_e': 'NAME_E',
   'NOM_F': 'NOM_F',
   'SOURCE': 'SOURCE',
   'PRV_CODE': 'PRV_CODE',
   'CARTO_UID': 'CARTO_UID',
   'type_e': 'TYPE_E',
   'TYPE_F': 'TYPE_F',
   'shape': 'Shape'},
  'oid_field_name': 'OBJECTID',
  'oid_property_name': 'objectid',
  '__orig_class__': archaic.info.Info[~T]})

In [4]:
class Mine:
    objectid: int
    name_e: str
    type_e: str
    shape: arcpy.Multipoint


feature_class = FeatureClass[Mine]("main.mines_pt")
feature_class.info.__dict__

{'model': __main__.Mine,
 'has_parameterless_constructor': True,
 'catalog_path': '.data\\canada.geodatabase\\main.mines_pt',
 'properties': {'objectid': 'OBJECTID',
  'name_e': 'NAME_E',
  'type_e': 'TYPE_E',
  'shape': 'SHAPE@'},
 'edit_properties': {'objectid': 'OBJECTID',
  'name_e': 'NAME_E',
  'type_e': 'TYPE_E',
  'shape': 'SHAPE@'},
 'oid_field_name': 'OBJECTID',
 'oid_property_name': 'objectid',
 '__orig_class__': archaic.info.Info[~T]}

In [5]:
feature_class = FeatureClass("main.glaciers_p")
feature_class.info.__dict__

{'model': types.SimpleNamespace,
 'has_parameterless_constructor': True,
 'catalog_path': '.data\\canada.geodatabase\\main.glaciers_p',
 'properties': {'OBJECTID': 'OBJECTID',
  'CARTO_UID': 'CARTO_UID',
  'FEATURE_E': 'FEATURE_E',
  'FEATURE_F': 'FEATURE_F',
  'Shape': 'Shape'},
 'edit_properties': {'OBJECTID': 'OBJECTID',
  'CARTO_UID': 'CARTO_UID',
  'FEATURE_E': 'FEATURE_E',
  'FEATURE_F': 'FEATURE_F',
  'Shape': 'Shape'},
 'oid_field_name': 'OBJECTID',
 'oid_property_name': 'OBJECTID',
 '__orig_class__': archaic.info.Info[~T]}